# How to build an AMBER-compatible host-guest complex using the SMIRNOFF force field starting with SYBYL-formatted `mol2` files

In [38]:
import parmed as pmd
import subprocess as sp
import numpy as np

from openeye import oechem
from openforcefield.typing.engines.smirnoff import *
from pdbfixer import PDBFixer

## Step 0: generate SYBYL-formatted `mol2` files

Starting with files containing AM1-BCC charges and GAFF v1.7 Lennard-Jones and bonded parameters:

```
antechamber -i bcd.mol2 -fi mol2 -o bcd-sybyl.mol2 -fo mol2 -at sybyl
antechamber -i ben.mol2 -fi mol2 -o ben-sybyl.mol2 -fo mol2 -at sybyl -dr n 
# Disable `acdoctor` to handle carboxylate group
```

In [39]:
path = './bcd-ben/'
bcd = path + 'bcd-sybyl.mol2'
ben = path + 'ben-sybyl.mol2'

In [40]:
!head -n 10 './bcd-ben/bcd.mol2' './bcd-ben/bcd-sybyl.mol2' 

==> ./bcd-ben/bcd.mol2 <==
@<TRIPOS>MOLECULE
Cpptraj generated mol2 file.
  147   154     7     0     0
SMALL
USER_CHARGES


@<TRIPOS>ATOM
      1 C1         16.6428   14.6231   33.9447 c3         1 MGO      0.304529
      2 H1         15.6508   14.1481   33.8157 h2         1 MGO      0.090782

==> ./bcd-ben/bcd-sybyl.mol2 <==
@<TRIPOS>MOLECULE
Cpptraj
  147   154     1     0     0
SMALL
No Charge or Current Charge


@<TRIPOS>ATOM
      1 C1          16.6430    14.6230    33.9450 C.3        1 MGO       0.304529
      2 H1          15.6510    14.1480    33.8160 H          1 MGO       0.090782


In [41]:
!head -n 10 './bcd-ben/ben.mol2' './bcd-ben/ben-sybyl.mol2'

==> ./bcd-ben/ben.mol2 <==
@<TRIPOS>MOLECULE
Cpptraj generated mol2 file.
   14    14     1     0     0
SMALL
USER_CHARGES


@<TRIPOS>ATOM
      1 C1         19.6968   19.9941   32.6877 ca         1 BEN     -0.161879
      2 H1         19.6968   19.9941   31.5977 ha         1 BEN      0.097073

==> ./bcd-ben/ben-sybyl.mol2 <==
@<TRIPOS>MOLECULE
Cpptraj
   14    14     1     0     0
SMALL
No Charge or Current Charge


@<TRIPOS>ATOM
      1 C1          19.6970    19.9940    32.6880 C.ar       1 BEN      -0.161879
      2 H1          19.6970    19.9940    31.5980 H          1 BEN       0.097073


## Step 1: load molecules in an `oechem.OEMol`

In [42]:
mols = []
ifs = oechem.oemolistream(bcd)
for mol in ifs.GetOEGraphMols():
    mols.append(oechem.OEMol(mol))

## Step 1.5: since `bcd` is a cyclic molecule, let's write a version without connectivity and then have OpenEye perceive connectivity based on [this issue](https://github.com/openforcefield/openforcefield/issues/66)

In [43]:
ofs = oechem.oemolostream()
ofs.open(path + 'bcd-no-conect.xyz')
flavor = oechem.OEOFlavor_Generic_Default
ofs.SetFlavor(oechem.OEFormat_XYZ, flavor)
oechem.OEWriteXYZFile(ofs, mols[0])
ofs.close()

In [44]:
ifs = oechem.oemolistream(path + 'bcd-no-conect.xyz')
for mol in ifs.GetOEGraphMols():
    oechem.OEDetermineConnectivity(mol)
    oechem.OEPerceiveBondOrders(mol)
    # Replace existing cyclodextrin
    mols[0] = oechem.OEMol(mol)

In [45]:
ifs = oechem.oemolistream(ben)
for mol in ifs.GetOEGraphMols():
    mols.append(oechem.OEMol(mol))

## Step 2: build reference molecules for water and ions

In [46]:
smiles = ['[Na+]', '[Cl-]', 'O']
for molecule in smiles:
    mol = oechem.OEMol()
    oechem.OESmilesToMol(mol, molecule)
    for atom in mol.GetAtoms():
        atom.SetPartialCharge(atom.GetFormalCharge())
    oechem.OEAddExplicitHydrogens(mol)
    oechem.OETriposAtomNames(mol)
    mols.append(mol)

## Step 3: read in fully solvated system with host, guest, water, and ions

### Step 3.1: create a PDB of the system with CONECT records
This requires using the "original" `mol2` files with GAFF atom names.

In [47]:
solvated_pdb = 'bcd-ben.pdb'
amber_prmtop = 'solvated.prmtop'
pdb_with_conect = 'bcd-ben-conect.pdb'
cpptraj_input = 'bcd-ben-conect.in'

cpptraj = \
'''
parm {}
trajin {}
trajout {} conect
'''.format(amber_prmtop, solvated_pdb, pdb_with_conect)

In [48]:
with open(path + cpptraj_input, 'w') as file:
    file.write(cpptraj)

In [49]:
cpptraj_output = sp.check_output(['cpptraj', '-i', cpptraj_input], cwd=path)

### Step 3.2: prune the CONECT records that correspond specifically to water molecules

In [50]:
first_water = sp.check_output(['grep', '-m 1', 'WAT', pdb_with_conect], cwd=path).decode("utf-8") 

In [51]:
first_water_residue = int(float(first_water.split()[1]))
print('First water residue = {}'.format(first_water_residue))

First water residue = 175


In [52]:
line_of_first_conect_to_delete = sp.check_output(['egrep', '-n', 'CONECT [ ]* {}'.format(str(first_water_residue)), 
                                                  pdb_with_conect], cwd=path).decode("utf-8")

In [53]:
line_to_delete_from = int(float(line_of_first_conect_to_delete.split(':')[0]))
print('Found first water CONECT entry at line = {}'.format(line_to_delete_from))

Found first water CONECT entry at line = 9192


In [54]:
truncated_file = sp.check_output(['awk', 'NR < {}'.format(line_to_delete_from), pdb_with_conect], cwd=path).decode("utf-8")

In [55]:
file = open(path + 'tmp.pdb', 'w')
file.write(truncated_file)
file.write("END")
file.close()
p = sp.check_output(['mv', 'tmp.pdb', '{}'.format(pdb_with_conect)], cwd=path)

In [56]:
pdb = PDBFixer(path + pdb_with_conect)

In [57]:
# residues = [r.name for r in pdb.topology.residues()]
# print(residues)

In [58]:
# names = [a.name for a in pdb.topology.atoms()]
# print(names)

## Step 4: create a ParmEd structure

In [59]:
ff = ForceField('forcefield/smirnoff99Frosst.ffxml', 'forcefield/tip3p.ffxml') 
system = ff.createSystem(pdb.topology, 
                         mols)
integrator = openmm.VerletIntegrator(1.0 * unit.femtoseconds)
context = openmm.Context(system, integrator)
context.setPositions(pdb.positions)

## Step 5: save the ParmEd structure as a `.prmtop` file

In [60]:
structure = pmd.openmm.topsystem.load_topology(pdb.topology, system, pdb.positions)

In [61]:
structure.save(path + 'bcd-ben-smirnoff.prmtop')

AttributeError: 'NoneType' object has no attribute 'used'

# Investigate whether the cyclodextrin `OEMol` is correct

In [ ]:
bcd = mols[0]

In [ ]:
from openeye.oedepict import *
OEPrepareDepiction(bcd)

In [ ]:
OERenderMolecule(path + "bcd.png", bcd)
from IPython.display import Image
Image(path + "bcd.png")

This seems okay.

In [62]:
structure.atoms

AtomList([
	<Atom C1 [0]; In MGO 0>
	<Atom H1 [1]; In MGO 0>
	<Atom O1 [2]; In MGO 0>
	<Atom C2 [3]; In MGO 0>
	<Atom H2 [4]; In MGO 0>
	<Atom O2 [5]; In MGO 0>
	<Atom HO2 [6]; In MGO 0>
	<Atom C3 [7]; In MGO 0>
	<Atom H3 [8]; In MGO 0>
	<Atom O3 [9]; In MGO 0>
	<Atom HO3 [10]; In MGO 0>
	<Atom C4 [11]; In MGO 0>
	<Atom H4 [12]; In MGO 0>
	<Atom C5 [13]; In MGO 0>
	<Atom H5 [14]; In MGO 0>
	<Atom O5 [15]; In MGO 0>
	<Atom C6 [16]; In MGO 0>
	<Atom H61 [17]; In MGO 0>
	<Atom H62 [18]; In MGO 0>
	<Atom O6 [19]; In MGO 0>
	<Atom HO6 [20]; In MGO 0>
	<Atom C1 [21]; In MGO 1>
	<Atom H1 [22]; In MGO 1>
	<Atom O1 [23]; In MGO 1>
	...
	<Atom H1 [6799]; In HOH 2229>
	<Atom H2 [6800]; In HOH 2229>
	<Atom O [6801]; In HOH 2230>
	<Atom H1 [6802]; In HOH 2230>
	<Atom H2 [6803]; In HOH 2230>
])

In [63]:
structure.bond_types

TrackedList([
	<BondType; k=340.000, req=1.090>
	<BondType; k=320.000, req=1.370>
	<BondType; k=310.000, req=1.526>
	<BondType; k=320.000, req=1.410>
	<BondType; k=553.000, req=0.960>
	<BondType; k=469.000, req=1.400>
	<BondType; k=367.000, req=1.080>
	<BondType; k=410.000, req=1.450>
	<BondType; k=656.000, req=1.250>
])

In [64]:
structure.bonds

TrackedList([
	<Bond <Atom H1 [175]; In HOH 21>--<Atom O [174]; In HOH 21>; type=None>
	<Bond <Atom H2 [176]; In HOH 21>--<Atom O [174]; In HOH 21>; type=None>
	<Bond <Atom H1 [178]; In HOH 22>--<Atom O [177]; In HOH 22>; type=None>
	<Bond <Atom H2 [179]; In HOH 22>--<Atom O [177]; In HOH 22>; type=None>
	<Bond <Atom H1 [181]; In HOH 23>--<Atom O [180]; In HOH 23>; type=None>
	<Bond <Atom H2 [182]; In HOH 23>--<Atom O [180]; In HOH 23>; type=None>
	<Bond <Atom H1 [184]; In HOH 24>--<Atom O [183]; In HOH 24>; type=None>
	<Bond <Atom H2 [185]; In HOH 24>--<Atom O [183]; In HOH 24>; type=None>
	<Bond <Atom H1 [187]; In HOH 25>--<Atom O [186]; In HOH 25>; type=None>
	<Bond <Atom H2 [188]; In HOH 25>--<Atom O [186]; In HOH 25>; type=None>
	<Bond <Atom H1 [190]; In HOH 26>--<Atom O [189]; In HOH 26>; type=None>
	<Bond <Atom H2 [191]; In HOH 26>--<Atom O [189]; In HOH 26>; type=None>
	<Bond <Atom H1 [193]; In HOH 27>--<Atom O [192]; In HOH 27>; type=None>
	<Bond <Atom H2 [194]; In HOH 27>--<A

In [68]:
bonds = structure.bonds

In [80]:
atoms = structure.atoms

In [90]:
for atom in atoms:
    if len(atom.bond_partners) == 0:
        print(atom, atom.bond_partners)
    if atom.bond_partners is None:
        print(atom, atom.bond_partners)

<Atom Na+ [161]; In Na+ 8> []
<Atom Na+ [162]; In Na+ 9> []
<Atom Na+ [163]; In Na+ 10> []
<Atom Na+ [164]; In Na+ 11> []
<Atom Na+ [165]; In Na+ 12> []
<Atom Na+ [166]; In Na+ 13> []
<Atom Na+ [167]; In Na+ 14> []
<Atom Cl- [168]; In Cl- 15> []
<Atom Cl- [169]; In Cl- 16> []
<Atom Cl- [170]; In Cl- 17> []
<Atom Cl- [171]; In Cl- 18> []
<Atom Cl- [172]; In Cl- 19> []
<Atom Cl- [173]; In Cl- 20> []


In [95]:
pmd.__version__

'2.7.3'